In [1]:
# first, we import the relevent packages.

# for data manipulation
import pandas as pd
import datetime as dt

# to scrape the web
from bs4 import BeautifulSoup as soupy
import requests
import time
import random

# dealing with consistent ssl error
import ssl

# for maths
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# for writing dictionaries and others as files
import json

In [2]:
# set the url for the site with the states
states_url = 'https://state.1keydata.com'

In [3]:
# make a connection to the page
states_page = requests.get(states_url, verify=False)

C:\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'state.1keydata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
# verify the connection
states_page.status_code

200

In [5]:
# get the actual html content
states_readable = soupy(states_page.content, 'html')

In [6]:
# now we need to initialize a list to store the states
# we initialize a list with 50 none values
# this makes it much faster to insert the value into the list
list_of_states = [None] * 50

In [7]:
# now we find all the href tags 
subset_states_readable = states_readable.find('div', {'id' : 'colwrap'})
href_all = subset_states_readable.find_all('a', href=True)

In [8]:
# then we get the text for each of the a tag values
for index, state in enumerate(href_all):
    value = state.text
    value = value.replace(' ', '_')
    list_of_states[index] = value

In [9]:
# we use a list comprehension to create the urls
list_of_states = [['https://en.wikipedia.org/wiki/List_of_United_States_representatives_from_' + a, a] for a in list_of_states]

In [10]:
# temporary ssl fix
ssl._create_default_https_context = ssl._create_unverified_context

In [11]:
# initialize a list for our pandas dataframes
# this is more efficient than appending multiple data frames
list_of_data_frames = [None] * 50
# get each table as a pandas data frame
# store it in the list
for index, (url, state) in enumerate(list_of_states):
    sleep = 4
    try:
        table = pd.read_html(url)[0].iloc[:,[0,1,2,3]]
        table.rename({table.columns[0]: 'Representative'}, axis = 'columns', inplace = True)
        for i in range(1,4):
            if 'Democratic' in table.iloc[:,i].values:
                table.rename({table.columns[i]: 'Party'}, axis = 'columns', inplace = True)
            elif ('1st' or 'At-large') in table.iloc[:,i].values:
                table.rename({table.columns[i]: 'District'}, axis = 'columns', inplace = True)
            else:
                table.rename({table.columns[i]: 'Years'}, axis = 'columns', inplace = True)
        table['State'] = state
        table[['Start', 'End']] = table['Years'].str.split('–' or '-', expand = True)
        table = table.drop(columns = 'Years')
        table['Start'] = table['Start'].str.strip()
        table['End'] = table['End'].str.strip()
        table['Start'] = pd.to_datetime(table['Start'], errors='coerce')
        table['End'] = table['End'].replace(['Present', 'present'], 'November 28, 2021')
        table['End'] = pd.to_datetime(table['End'], errors='coerce')
        list_of_data_frames[index] = table
        time.sleep(random.uniform(0,sleep))
    except:
        time.sleep(random.uniform(0,sleep))
        pass

In [12]:
res = [i for i in range(len(list_of_data_frames)) if list_of_data_frames[i] is None]

In [13]:
# this is a list of indexes where we did not store a dataframe
# we will have to go back and scrape these values individually
res

[1, 7, 9, 10, 15, 18, 27, 49]

In [14]:
skel = pd.concat(list_of_data_frames, axis = 0, ignore_index = True)

In [15]:
skel = skel[skel['Start'].isnull() == False]

In [16]:
skel = skel[skel['End'].isnull() == False]

In [17]:
skel['Start_Year'] = skel['Start'].dt.year

In [18]:
skel['End_Year'] = skel['End'].dt.year

In [19]:
skel['Duration'] = skel['End'] - skel['Start']

In [20]:
skel['Duration_simple'] = pd.Series(skel['Duration']).dt.days/365

In [21]:
dict_1 = {}
year_init = 1789

In [22]:
for i in range(1, 117):
    dict_1[i] = year_init
    year_init = year_init + 2

In [23]:
for congress, year in dict_1.items():
    skel[str(congress)] = ""

C:\Users\cmacd\AppData\Local\Temp/ipykernel_2256/2980589466.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  skel[str(congress)] = ""


In [24]:
for index, row in skel.iterrows():
    for congress, year in dict_1.items():
        if row['Start'].year <= year:
            if row['End'].year > (year + 1):
                skel.at[index,str(congress)] = 1
            else:
                skel.at[index,str(congress)] = 0
        else:
            skel.at[index,str(congress)] = 0

In [25]:
list_of_dfs = []

In [26]:
relevant_congresses = list(dict_1.keys())
relevant_congresses = [str(x) for x in relevant_congresses]

In [27]:
for congress, year in dict_1.items():
    if (year >=1960) and (year<2019):
        df = skel[skel[str(congress)] == 1.0].copy(deep=True)
        df['Congress'] = congress
        df['Congress_Start'] = year
        df['Congress_End'] = year + 2
        df = df.drop(columns = relevant_congresses)
        df = df.reset_index()
        
        list_of_dfs.append(df)

In [28]:
skel = pd.concat(list_of_dfs, ignore_index = True)

In [29]:
skel = skel.drop(columns = ['index'])
skel['Incumbancy'] = skel['Congress_Start'].astype(int) - skel['Start_Year'].astype(int)

In [30]:
skel

,Representative,Party,District,State,Start,End,Start_Year,End_Year,Duration,Duration_simple,Congress,Congress_Start,Congress_End,Incumbancy
0,George W. Andrews,Democratic,3rd,Alabama,1944-03-14,1963-01-03,1944,1963,6869 days,18.819178,87,1961,1963,17
1,Frank W. Boykin,Democratic,1st,Alabama,1935-07-30,1963-01-03,1935,1963,10019 days,27.449315,87,1961,1963,26
2,Carl Elliott,Democratic,7th,Alabama,1949-01-03,1963-01-03,1949,1963,5113 days,14.008219,87,1961,1963,12
3,George M. Grant,Democratic,2nd,Alabama,1938-06-14,1963-01-03,1938,1963,8969 days,24.572603,87,1961,1963,23
4,George Huddleston Jr.,Democratic,9th,Alabama,1955-01-03,1963-01-03,1955,1963,2922 days,8.005479,87,1961,1963,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11636,Ron Kind,Democratic,3rd,Wisconsin,1997-01-03,2021-11-28,1997,2021,9095 days,24.917808,115,2017,2019,20
11637,Gwen Moore,Democratic,4th,Wisconsin,2005-01-03,2021-11-28,2005,2021,6173 days,16.912329,115,2017,2019,12
11638,Mark Pocan,Democratic,2nd,Wisconsin,2013-01-03,2021-11-28,2013,2021,3251 days,8.906849,115,2017,2019,4
11639,Paul Ryan,Republican,1st,Wisconsin,1999-01-03,2019-01-03,1999,2019,7305 days,20.013699,115,2017,2019,18


In [31]:
skel.to_csv('main_frame.csv', encoding='utf-8', index=False)